In [43]:
import pandas as pd
import numpy as np
import re

import sys; sys.path.append("../")
from config import CredentialHandler

auth = CredentialHandler("../auth.txt")

Succesfully loaded credentials


In [44]:
url='https://en.wikipedia.org/wiki/List_of_cryptocurrencies'

currency_list = pd.read_html(url, header=0)[0]
currency_list = currency_list[["Currency", "Symbol", "Founder(s)"]]
currency_list = currency_list[currency_list.Symbol != 'TIT']

remove_citations = lambda x: re.sub("([\(\[]).*?([\)\]])", "\g<1>\g<2>", str(x)).replace('[]', '').replace('()', '')

for col in list(currency_list.columns):
    currency_list[col] = currency_list[col].apply(remove_citations)
    
currency_list['refs'] = currency_list['Currency'].apply(lambda x: [x.lower().replace('coin', '')])

symbol_refs = iter(currency_list['Symbol'].apply(lambda x: x.lower().split(', ')))
currency_list['refs'].apply(lambda x: [x.append(n) for n in next(symbol_refs)])
currency_list.loc[18, 'refs'] = ['ether', 'ethereum', 'eth']

currency_list = currency_list.rename(columns={"Founder(s)":'founder'})


flat_currency_list = currency_list.refs.apply(pd.Series).unstack().reset_index().dropna()
flat_currency_list = currency_list.reset_index().merge(flat_currency_list, left_on='index', right_on='level_1')

flat_currency_list = flat_currency_list[['Currency', 'Symbol', 0]].rename(columns={'Currency':'currency', 'Symbol':'symbol', 0:'ref'})
flat_currency_list




,currency,symbol,ref
0,Bitcoin,"BTC, XBT, ₿",bit
1,Bitcoin,"BTC, XBT, ₿",btc
2,Bitcoin,"BTC, XBT, ₿",xbt
3,Bitcoin,"BTC, XBT, ₿",₿
4,Litecoin,"LTC, Ł",lite
5,Litecoin,"LTC, Ł",ltc
6,Litecoin,"LTC, Ł",ł
7,Namecoin,NMC,name
8,Namecoin,NMC,nmc
9,Peercoin,PPC,peer


In [45]:
currency_list.to_hdf("data/currency_list.h5", key='s')

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:2505: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['Currency', 'Symbol', 'founder', 'refs'], dtype='object')]

  encoding=encoding,


In [48]:
import tweepy

#pass twitter credentials to tweepy
twit_auth = tweepy.OAuthHandler(auth._twitter_key, 
                                auth._twitter_secret)
twit_auth.set_access_token(auth._twitter_access_key, 
                           auth._twitter_access_secret)

api = tweepy.API(twit_auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

tracked_users = pd.DataFrame({"ID":['4194920189', 
                                    '14338147', 
                                    '1285360265450618886', 
                                    "44196397", 
                                    "961445378",
                                    "295218901",
                                    "176758255",
                                    "396045469",
                                    "143053926",
                                    "1469101279",
                                    "3087276349",
                                    "4826176219",
                                    "2176983931",
                                    "3022775424",
                                    "15506669"
                                    ]})

retrieve_username = lambda id_: api.get_user(id_)._json['name']

tracked_users['name'] = tracked_users.ID.apply(retrieve_username)

tracked_users

,ID,name
0,4194920189,Satoshi Nakamoto
1,14338147,Charlie Lee [LTC⚡]
2,1285360265450618886,Dogecoin
3,44196397,Elon Musk
4,961445378,John McAfee
5,295218901,vitalik.eth
6,176758255,Roger Ver
7,396045469,Barry Silbert
8,143053926,Gavin Andresen
9,1469101279,"Andreas (BEWARE of ""giveaway"" scams!)"


In [49]:
tracked_users.to_hdf("data/tracked_users.h5", key='users')
currency_list.to_hdf("data/currency_list.h5", key='currencies')
flat_currency_list.to_hdf("data/flat_currency_list.h5", key='currencies')

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:2505: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['Currency', 'Symbol', 'founder', 'refs'], dtype='object')]

  encoding=encoding,
